In [1]:
import numpy as np
import pandas as pd
import warnings
from math import isnan
import re

warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../Output/0504/train_data_tezheng1.csv', encoding='utf-8')
test = pd.read_csv('../Output/0504/test_data_tezheng1.csv', encoding='utf-8')

In [3]:
train.shape

(38199, 399)

In [4]:
# 将训练集中指标为NAN的舍弃掉

train = train[~train.收缩压.isnull()]
train = train[~train.舒张压.isnull()]

In [5]:
train.shape

(38192, 399)

In [6]:
num_cols = list(train.describe().columns[5:])   # 纯数字+NAN

In [12]:
# 查找混合型字段
# 如 >100次/分,窦性心动过速    ------简单混合型
has_num = []
p = re.compile('\d+\.?\d*')
r = train.shape[0]
for c in train.columns:
    nulls = train[c].isnull().sum()
    a = train[c].apply(lambda x: 1 if len(p.findall(str(x))) != 0 else 0).sum()
    if (a+nulls) > r*0.9:
        has_num.append(c)
has_num

['vid',
 '收缩压',
 '舒张压',
 '血清甘油三酯',
 '血清高密度脂蛋白',
 '血清低密度脂蛋白',
 '004997',
 '0104',
 '0125',
 '0130',
 '0131',
 '0218',
 '0424',
 '0437',
 '0438',
 '0439',
 '0441',
 '0449',
 '0732',
 '0986',
 '100005',
 '100006',
 '100007',
 '100012',
 '100013',
 '100014',
 '10002',
 '10003',
 '10004',
 '10009',
 '1104',
 '1106',
 '1107',
 '1110',
 '1112',
 '1115',
 '1117',
 '1124',
 '1125',
 '1127',
 '1319',
 '1320',
 '1321',
 '1322',
 '1325',
 '1326',
 '1331',
 '1334',
 '1345',
 '139',
 '143',
 '1474',
 '155',
 '1814',
 '1815',
 '183',
 '1840',
 '1844',
 '1845',
 '1850',
 '1873',
 '189',
 '190',
 '191',
 '192',
 '193',
 '20002',
 '2174',
 '2177',
 '2282',
 '2333',
 '2371',
 '2372',
 '2376',
 '2386',
 '2390',
 '2403',
 '2404',
 '2405',
 '2406',
 '2407',
 '2409',
 '2420',
 '269003',
 '269004',
 '269005',
 '269006',
 '269007',
 '269008',
 '269009',
 '269010',
 '269011',
 '269012',
 '269013',
 '269014',
 '269015',
 '269016',
 '269017',
 '269018',
 '269019',
 '269020',
 '269021',
 '269022',
 '269023',
 '269

In [13]:
len(has_num)

251

In [15]:
print (train['0104'].unique())
print (len(train['0104'].unique()))
print (train['0104'].isnull().sum())

[nan 30.0 28.0 31.0 26.0 29.0 33.0 32.0 37.0 34.0 22.0 27.0 25.0 19.0 23.0
 35.0 24.0 '27.0' '37.0' '30.0' '31.0' '34.0' '28.0' '26.0' '25.0' '29.0'
 '23.0' '28' '22.8' '53.0' '32.0' '24.0' '35.0' '26' '33.0' '29' '22.0'
 '心内各结构未见明显异常' 39.0 1.87 20.0 2.73 '20.0' '38.0' '25' '21.0' '19' '21'
 '29.1' '31.5' '24' '20' '16.0' 21.0 28.1 25.4 34.2 25.8 30.5 19.6 30.8
 30.1 29.3 21.8 23.8 33.4 31.8 15.7 26.5 29.7 16.0 17.0 18.0 24.9 24.5 24.6
 26.8 29.9 31.5 '26.2' '28.9' '26.9' '27' '36.0' '25.8' '27.8' '40.9'
 '29.3' 30.6 28.2 38.0 36.0 31.9 28.8 29.1 27.8 29.5 '18.0' '27.2' '19.0'
 28.6 28.4 28.3 '29.7' '32' '28.1' '17.0' '18' '30' '23' 228.0 0.0 '10.0'
 '15.0' '33' '29.8' '25.7']
117
36457


In [ ]:
train['10002']

In [7]:
main_cols = ['vid','收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']  # vid和指标字段
digit_cols1 = ['1814', '190', '191', '2403', '2404', '2405']                                    # BMI指标和数字（含未查等特殊）
digit_cols2 = ['10004','10002','1115','1117','1815','183','1850','192','193','2174','314']      # 纯数字型 （float+str）
wenben_cols = ['0409', '0434', '0424', 'B701','A801','A705', '2302', '0114','3401','1001','0102','A202']  # 文本
emu_cols = ['3190', '3191', '3192', '3195', '3196','3197','3430']                               # 枚举型
num_and_wenben_cols = ['3193','1840','A701','A703']                                             # 数字型 （含>= 阴性等特殊）
select_cols = digit_cols1 + wenben_cols + num_cols + main_cols + num_and_wenben_cols + emu_cols + digit_cols2

3402基本无异常

A601大部分详见纸质报告

数字为主： A701、A703、3193、1840  

性别标签: 0120 0121 0125 0130

In [8]:
train1 = train[[c for c in train.columns if c in select_cols]]
test1 = test[[c for c in test.columns if c in select_cols]]

In [9]:
def numAndWenben(x):
    x = str(x)
    if x == 'nan' or x =='阴性' or x=='-' or x=='未做':
        return np.nan
    else:
        p = re.compile('\d+\.?\d*')
        return p.findall(x)[0]

def processNumAndWenben(train):
    data = train.copy()
    cols = ['3193','1840','A701','A703']
    for c in cols:
        data[c] = data[c].apply(numAndWenben).astype('float').fillna(0)
    return data

test2 = processNumAndWenben(test1)
train2 = processNumAndWenben(train1)

In [10]:
def label_Bchao(x):
    x = str(x)
    if '绝经' in x:
        return 0
    if '高血压' in x:
        return 1
    if '脂肪肝' in x:
        return 2
    else:
        return 3
    
def processBchao(data):
    train = data.copy()
    cols = ['0102','A202']
    for col in cols:
        train[col+'bchao'] = train[col].apply(label_Bchao)
    return train.drop(cols, axis=1)

test3 = processBchao(test2)
train3 = processBchao(train2)

In [11]:
# 枚举型'3190', '3191', '3192', '3195', '3196','3197',3430
def emuFeature(x):
    good_labels = ['nan','阴性','未做','normal','正常']
    nor_labels = ['+-','未见']
    bad_labels = ['阳性','阳性(+)','阳性(1+)','阳性3+']
    x = str(x)
    if x in nor_labels or '+-' in x:
        return 1
    elif x in good_labels or '-' in x:
        return 0
    elif x in bad_labels or '+' in x:
        return 2
    else:
        return 0
    
def processEmu(data):
    train = data.copy()
    for col in emu_cols:
        train[col+'_emu'] =  train[col].apply(emuFeature)
        
    return train.drop(emu_cols, axis=1)

test4 = processEmu(test3)
train4 = processEmu(train3)

In [12]:
def xindiantu(x):
    x = str(x)
    if x == 'nan' or '正常心电图' in x:
        return 0
    return 1

def process1001(data):
    train = data.copy()
    train['1001_xindiantu'] = train['1001'].apply(xindiantu)
    return train.drop(['1001'], axis=1)

test5 = process1001(test4)
train5 = process1001(train4)

枚举型变量：

'3190', '3191', '3192', '3195', '3196','3197'   ：阴阳性，+-

混合型变量：

 '0424','1840','3193'

In [13]:
def getFeature3401(x):
    x = str(x)
    if x == 'nan' or ('未见明显异常' in x and '检查' not in x and '复查'not in x):
        return 0
    elif '退变' in x :
        return 2
    elif '未见明显异常' in x and ('检查'  in x or '复查' in x):
        return 1
    else:
        return 3

def process3401(data):
    train = data.copy()
    train['3401_ct'] = train['3401'].apply(getFeature3401)
    return train.drop(['3401'], axis=1)

test6 = process3401(test5)
train6 = process3401(train5)

In [14]:
def process409_434(data):
    train = data.copy()
    cols = []
    for c in ['0409', '0434']:
        train[c] = train[c].astype('str')
        if c == '0434':
            train[c+'_smoke'] = train[c].apply(lambda x: '吸烟' in  x).astype('int')
            train[c+'_gaoxueya'] = train[c].apply(lambda x: '高血压' in  x or '血压偏高' in x).astype('int')
            train[c+'_gaoxuezhi'] = train[c].apply(lambda x: '血脂偏高' in  x ).astype('int')
            train[c+'_tangniaobing'] = train[c].apply(lambda x: '糖尿病' in  x ).astype('int')
            train[c+'_niaosuan'] = train[c].apply(lambda x: '尿酸偏高' in  x ).astype('int')
            train[c+'_gan'] = train[c].apply(lambda x: '肝' in  x).astype('int')
            train[c+'_zhifangan'] = train[c].apply(lambda x: '脂肪肝' in  x).astype('int')
            train[c+'_feipang'] = train[c].apply(lambda x: '肥胖' in  x).astype('int')
            train[c+'_heart'] = train[c].apply(lambda x: '心' in  x or '冠状动脉' in x).astype('int')
            train[c+'_pregnancy'] = train[c].apply(lambda x: '孕' in  x and '宫外孕术后' not in x).astype('int')
            train[c+'_yanzheng'] = train[c].apply(lambda x: '炎' in  x).astype('int')
            train[c+'_jzxkangjin'] = train[c].apply(lambda x: '甲亢' in  x or '甲状腺功能亢进'in x).astype('int')
            train[c+'_jzxjt'] = train[c].apply(lambda x: '甲状腺功能减退' in  x).astype('int')
        else:
            train[c+'_free'] = train[c].apply(lambda x: '未发现明显异常' in  x or '未见异常' in x).astype('int')
            train[c+'_yanzheng'] = train[c].apply(lambda x: '炎' in  x).astype('int')
            train[c+'_niaosuan'] = train[c].apply(lambda x: '高尿酸' in  x ).astype('int')
            train[c+'_gaoxuezhi'] = train[c].apply(lambda x: '血脂偏高' in  x ).astype('int')
            train[c+'_gaoxuetang'] = train[c].apply(lambda x: '血糖偏高' in  x ).astype('int')
            train[c+'_gaoxueya'] = train[c].apply(lambda x: '高血压' in  x or '血压偏高' in x).astype('int')
            train[c+'_gaoxuezhi'] = train[c].apply(lambda x: '血脂偏高' in  x or '高血脂' in x).astype('int')
            train[c+'_zhifangan'] = train[c].apply(lambda x: '脂肪肝' in  x).astype('int')
            train[c+'_beatslow'] = train[c].apply(lambda x: '心动过缓' in  x).astype('int')
            train[c+'_beatfast'] = train[c].apply(lambda x: '心动过速' in  x).astype('int')
            train[c+'_beatunregular'] = train[c].apply(lambda x: '心律不齐' in  x).astype('int')
            train[c+'_beatfre'] = train[c].apply(lambda x: '偶发早搏' in  x).astype('int')
            train[c+'_heartok'] = train[c].apply(lambda x: '心率正常' in  x).astype('int')
            train[c+'_heartnoise'] = train[c].apply(lambda x: '心脏杂音' in  x or '心音' in x).astype('int')
            train[c+'_tangniaobing'] = train[c].apply(lambda x: '糖尿病' in  x ).astype('int')
            train[c+'_gan'] = train[c].apply(lambda x: '肝' in  x).astype('int')
            train[c+'_heart'] = train[c].apply(lambda x: '心肌' in  x or '心脏' in x).astype('int')
            train[c+'_jzxkangjin'] = train[c].apply(lambda x: '甲亢' in  x or '甲状腺功能亢进'in x).astype('int')
            train[c+'_jzxjt'] = train[c].apply(lambda x: '甲状腺功能减退' in  x).astype('int')
            train[c+'_tangniaobing'] = train[c].apply(lambda x: '糖尿病' in  x ).astype('int')

    return train.drop(['0409','0434'], axis=1)

test7 = process409_434(test6)
train7 = process409_434(train6)

In [15]:
def get_heart_rate(x):
    import re
    result = re.findall(r"\d+",x)
    if result != []:
        return result[0]
    else :
        return 0

def is_heart_slow(x):
    return ('过缓' in x) or ('<' in x )or (int(get_heart_rate(x))<60 and int(get_heart_rate(x))>1)

def is_heart_fast(x):
    return ('过速' in x) or ('>' in x )or (int(get_heart_rate(x))>100 ) 

def is_heart_normal(x):
    return (int(get_heart_rate(x))>60 and int(get_heart_rate(x))<100) or ('正常' in x) or ('未见' in x)

def process424(data):
    train = data.copy()
    train['0424'] = train['0424'].astype('str')
    train['424_heart_rate'] = (train['0424'].apply(lambda x: get_heart_rate(x))).astype('int')
    train['424_heart_slow'] = train['0424'].apply(lambda x: is_heart_slow(x)).astype('int')
    train['424_heart_fast'] = train['0424'].apply(lambda x: is_heart_fast(x)).astype('int')
    train['424_heart_normal'] = train['0424'].apply(lambda x: is_heart_normal(x)).astype('int')

    return train.drop(['0424'], axis=1)

test8 = process424(test7)
train8 = process424(train7)

In [16]:
test8.shape

(9538, 169)

In [17]:
def process114_2302(data):
    train = data.copy()
    train['0114_normal'] = (train['0114'].fillna(-1)).apply(lambda x: int('胆囊大小、形态正常' in  x) if x!= -1 else x)
    train = train.drop(['0114'], axis=1)

    train['2302_jiankang'] = (train['2302'].fillna(-1)).apply(lambda x: int('健康' in  x) if x!= -1 else x)
    train = train.drop(['2302'], axis=1)
    return train

test9 = process114_2302(test8)
train9 = process114_2302(train8)

In [18]:
def processAB(data):
    train = data.copy()
    train['A705_label'] = (train['A705'].fillna(-1)).apply(lambda x: int('数字化肝超未发现明显异常' in  x) if x != -1 else x)
    train['A801_label'] = (train['A801'].fillna(-1)).apply(lambda x: int('脑血流图未发现明显异常' in  x) if x != -1 else x)
    train['B701_label'] = (train['B701'].fillna(-1)).apply(lambda x: int('糖尿病或糖尿病前期风险较小' in  x) if x != -1 else x)
    train = train.drop(['B701','A801','A705'], axis=1)
    return train

test10 = processAB(test9)
train10 = processAB(train9)

纯数字table_id：
1814
190
191
2403
2404
2405

'10002','10004','1115', '1117','1815','183','1840','1850'', '192', '193', '2174',  '314'

In [19]:
def modifyYiChang(x):
    if ';' in str(x):
        spices = x.split(';')
        if spices.count('nan') == 2 or spices.count('未查') == 2 or spices.count('弃查') == 2:
            return np.nan
        elif spices.count('nan') == 1:
            return spices[1-spices.index('nan')]
        elif spices.count('未查') == 1:
            return spices[1-spices.index('未查')]
        elif spices.count('弃查') == 1:
            return spices[1-spices.index('弃查')]
        else :
            return spices[0]
    elif '查' in str(x):
        return np.nan
    return x

def process4(data):
    train = data.copy()
    for col in digit_cols1:
        train[col] = train[col].apply(modifyYiChang).astype('float').fillna(0)
    return train

test11 = process4(test10)
train11 = process4(train10)

In [20]:
def processPureDigit(data):
    train = data.copy()
    for col in num_cols:
        train[col] = train[col].apply(modifyYiChang).astype('float').fillna(0)
    return train

test12 = processPureDigit(test11)
train12 = processPureDigit(train11)

In [21]:
def searchDigitRe(x):
    x = str(x)
    if x == 'nan':
        return 0
    else:
        try:
            p = re.compile('\d+\.?\d*')
            return p.findall(x)[0]
        except:
            return 0

def processDigit2(data):
    train = data.copy()
    for col in digit_cols2:
        train[col] = train[col].apply(searchDigitRe).astype('float').fillna(0)
    return train

test_end = processDigit2(test12)
train_end = processDigit2(train12)

In [22]:
train_end.to_csv('../Output/0504/train_data_tezheng2.csv', index=False, encoding='utf-8')
test_end.to_csv('../Output/0504/test_data_tezheng2.csv', index=False, encoding='utf-8')

In [23]:
train_end.shape

(38192, 169)

————————————————————————————————————————————————————————————

In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")


train1 = pd.read_csv('../Output/0504/train_data_tezheng2.csv', encoding='utf-8')
test1 = pd.read_csv('../Output/0504/test_data_tezheng2.csv', encoding='utf-8')

In [24]:
train1 = train_end
test1 = test_end


In [2]:
train1.shape

(38192, 169)

In [25]:
# 比赛要求自定义log_loss
def cal_logloss(label, pre_label):
    rows = len(label)
    log_sum = sum((np.log(pre_label+1) - np.log(label+1)) ** 2)
    return log_sum / rows


In [26]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [27]:
digit_cols = num_cols + ['424_heart_rate','1814', '190', '191', '2403', '2404', '2405'] + digit_cols2 + num_and_wenben_cols

In [28]:
# 归一化
rows = train1.shape[0]
data = train1.append(test1)
data[['424_heart_rate','1814', '190', '191', '2403', '2404', '2405']] = data[['424_heart_rate','1814', '190', '191', '2403', '2404', '2405']].fillna(0)
for c in digit_cols:
    data[c] = MinMaxScaler().fit_transform(data[c].values.reshape(-1, 1))

In [29]:
train1 = data[:rows]
test1 = data[rows:]
train1.shape

(38192, 169)

In [30]:
shousuoya_label = train1.pop('收缩压')
shuzhangya_label = train1.pop('舒张压')
highzhidanbai_label = train1.pop('血清高密度脂蛋白')
ganyou_label = train1.pop('血清甘油三酯')
lowzhidanbai_label = train1.pop('血清低密度脂蛋白')

train1 = train1.drop('vid', axis =1)

In [31]:
test1.pop('收缩压')
test1.pop('舒张压')
test1.pop('血清高密度脂蛋白')
test1.pop('血清甘油三酯')
test1.pop('血清低密度脂蛋白')

test1_vid = test1.pop('vid')

In [32]:
train_table = {'收缩压':shousuoya_label,
              '舒张压':shuzhangya_label,
              '血清甘油三酯':ganyou_label,
              '血清高密度脂蛋白':highzhidanbai_label,
              '血清低密度脂蛋白':lowzhidanbai_label}

In [33]:
LGBM = lgb.LGBMRegressor(objective='regression', 
                         max_depth=7, 
                         n_estimators=600, 
                         learning_rate=0.01, 
                         num_leaves=63, 
                         min_samples_split=2)

In [ ]:
train1.columns

In [34]:
drop_cols = []
log_score = 0
train2 = train1[[c for c in train1.columns if c not in drop_cols]]
for key in train_table.keys():
    X_train,X_test,y_train,y_test = train_test_split(train2, train_table[key], test_size=0.05, random_state=42)
    LGBM.fit(X_train, y_train)
    preLabel = LGBM.predict(X_test)
    log_cur = cal_logloss(y_test, preLabel)
    log_score += log_cur
    print (log_cur)
print (log_score / 5)

0.0147122887927
0.0184465667067
0.0846061387269
0.0140281910554
0.0376471606431
0.033888069185


In [35]:
test_label = {}
for key in train_table.keys():
    LGBM.fit(train1, train_table[key])
    preLabel = LGBM.predict(test1)
    test_label[key] = preLabel


In [36]:
def generateResult(vid, shousuoya_label, shuzhangya_label, ganyou_label, highzhidanbai_label, lowzhidanbai_label, filename='../Output/result0504.csv'):
    result = pd.DataFrame({'vid':vid})
    result['收缩压'] = shousuoya_label
    result['舒张压'] = shuzhangya_label
    result['血清甘油三酯'] = ganyou_label
    result['血清高密度脂蛋白'] = highzhidanbai_label
    result['血清低密度脂蛋白'] = lowzhidanbai_label
    result.to_csv(filename, index=False, header=False ,line_terminator='\r')

In [37]:
generateResult(test1_vid, test_label['收缩压'], test_label['舒张压'], 
               test_label['血清甘油三酯'], test_label['血清高密度脂蛋白'], 
               test_label['血清低密度脂蛋白'])